In [20]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
from tqdm import tqdm
import os
import re
import pandas as pd

dataset_path = 'DDS_Data_SEU'

PGB_path = '/Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB'
RGB_path = '/Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/RGB'

In [8]:
def extract_fault(file_name):
    match = re.search(r'\d+', file_name)
    if match:
        return int(match.group(0)[0])  # Extract the first digit
    else:
        return None

def process_files_to_csv(data_folders, output_file):
    total_files = sum([len(files) for data_folder in data_folders for r, d, files in os.walk(data_folder)])

    with open(output_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5', 'Channel6', 'Channel7', 'Channel8', 'Speed', 'Type', 'Fault'])

        with tqdm(total=total_files, desc="Processing files", unit="file") as pbar:
            for data_folder in data_folders:
                for root, dirs, files in os.walk(data_folder):
                    for file in files:
                        if file.endswith('.txt'):
                            file_path = os.path.join(root, file)
                            path_parts = file_path.split('/')
                            type = path_parts[-3] if path_parts[-3] in ['PGB', 'RGB'] else None
                            if type is not None:
                                speed = path_parts[-2] if path_parts[-2] != 'Variable_speed' else path_parts[-1]
                                speed = 'Variable' if 'Experiment' in speed else speed.replace('Hz-', '_').replace('Hz', '')
                                fault = extract_fault(file) if 'Experiment' not in file else file.split('_')[0]

                                data = pd.read_csv(file_path, sep='\t', encoding='ISO-8859-1')
                                reshaped_data = data.values[:, :]

                                for row_data in tqdm(reshaped_data, desc="Processing rows", unit="row", leave=False):
                                    row = row_data.tolist() + [speed, type, fault]
                                    csv_writer.writerow(row)
                            pbar.update()

In [9]:
# Example usage
process_files_to_csv([PGB_path, RGB_path], '/Volumes/DataBoi/DDS_paper/data_new.csv')

Processing files:  99%|█████████▉| 342/344 [38:13<00:13,  6.71s/file] 


In [35]:
def extract_fault(file_name):
    match = re.search(r'\d+', file_name)
    if match:
        return int(match.group(0)[0])  # Extract the first digit
    else:
        return None

def process_files_to_csv_variable(data_folders, output_file):
    total_files = sum([len(files) for data_folder in data_folders for r, d, files in os.walk(data_folder) if '.ipynb_checkpoints' not in r])

    with open(output_file, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5', 'Channel6', 'Channel7', 'Channel8', 'Speed', 'Type', 'Fault'])

        with tqdm(total=total_files, desc="Processing files", unit="file") as pbar:
            for data_folder in data_folders:
                for root, dirs, files in os.walk(data_folder):
                    if '.ipynb_checkpoints' in root:
                        continue  # Skip .ipynb_checkpoints folders
                    for file in files:
                        file_path = os.path.join(root, file)
                        if file.endswith('.txt'):
                            path_parts = file_path.split('/')
                            type = path_parts[-4] if path_parts[-4] in ['PGB', 'RGB'] else None  # Check type
                            speed = path_parts[-3]  # Speed will always be the third last element in the path
                            if type is not None and speed is not None:
                                fault = extract_fault(file) if 'Experiment' not in file else file.split('_')[0]

                                try:
                                    data = pd.read_csv(file_path, sep='\t', encoding='ISO-8859-1')
                                    reshaped_data = data.values[:, :]
                                except Exception as e:
                                    print(f'Error reading file: {file_path}. Error: {e}')
                                    continue

                                for row_data in tqdm(reshaped_data, desc="Processing rows", unit="row", leave=False):
                                    row = row_data.tolist() + [speed, type, fault]
                                    csv_writer.writerow(row)
                            else:
                                print(f'Skipped file (not correct type or speed): {file_path}')
                        else:
                            print(f'Skipped file (not .txt): {file_path}')
                    pbar.update()


In [36]:
# Example usage
process_files_to_csv_variable([PGB_path, RGB_path], '/Volumes/DataBoi/DDS_paper/data_new_variable.csv')

Processing files:   0%|          | 0/344 [00:00<?, ?file/s]

Skipped file (not .txt): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/.DS_Store
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/50_0/7Inner_50_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/50_0/4Surface_50_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/50_0/6Combination_50_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/50_0/1Chipped_50_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/50_0/2Miss_50_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/50_0/0Health_50_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facult

Processing files:   5%|▌         | 18/344 [01:23<41:09,  7.57s/file]

Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/30_0/0Health_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/30_0/2Miss_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/30_0/1Chipped_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/30_0/7Inner_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/30_0/4Surface_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/30_0/6Combination_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/PGB/30_0/3Root_30_0_1-10.txt
Skipped file (not correct type or speed): /Use

Processing files:  12%|█▏        | 42/344 [02:49<20:16,  4.03s/file]


Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/RGB/30_0/0Health_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/RGB/30_0/2Miss_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/RGB/30_0/1Chipped_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/RGB/30_0/7Inner_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/RGB/30_0/4Surface_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/RGB/30_0/6Combination_30_0_1-10.txt
Skipped file (not correct type or speed): /Users/eduard.hogea/Documents/Facultate/DDS_Paper/DDS_Data_SEU/RGB/30_0/3Root_30_0_1-10.txt
Skipped file (not correct type or speed): /Use

# Data preprocessing

In [14]:
import pandas as pd
from tqdm import tqdm

# Initialize a dictionary to store the fault counts
fault_counts = {}

# Specify the CSV file path
csv_file = '/Volumes/DataBoi/DDS_paper/data_new.csv'

# Set the chunk size for reading the CSV
chunk_size = 100000  # Adjust the chunk size according to your memory limitations

# Iterate through the CSV file using chunksize
with tqdm(total=1, unit='chunk', desc='Processing CSV') as pbar:
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        
        print(chunk)
        # Assuming there is a column named 'fault' in the CSV representing the fault type
        fault_column = 'Fault'

        # Count the occurrences of each fault in the current chunk
        fault_chunk_counts = chunk[fault_column].value_counts()

        # Aggregate the counts with the overall fault_counts dictionary
        for fault, count in fault_chunk_counts.items():
            fault_counts[fault] = fault_counts.get(fault, 0) + count

        pbar.update()

# Print the fault counts
for fault, count in fault_counts.items():
    print(f"Fault: {fault}, Count: {count}")


In [37]:
import pandas as pd
from tqdm import tqdm

# Initialize dictionaries to store the counts
speed_counts = {}
type_counts = {}

# Specify the CSV file path
csv_file = '/Volumes/DataBoi/DDS_paper/data_new_variable.csv'

# Set the chunk size for reading the CSV
chunk_size = 1000000  # Adjust the chunk size according to your memory limitations

# Iterate through the CSV file using chunksize
with tqdm(total=1, unit='chunk', desc='Processing CSV') as pbar:
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
        # Assuming there is a column named 'Speed' in the CSV representing the speed values
        
        speed_column = 'Speed'

        # Count the occurrences of each speed value in the current chunk
        speed_chunk_counts = chunk[speed_column].value_counts()

        # Aggregate the counts with the overall speed_counts dictionary
        for speed, count in speed_chunk_counts.items():
            speed_counts[speed] = speed_counts.get(speed, 0) + count

        # Assuming there is a column named 'Type' in the CSV representing the types
        type_column = 'Type'

        # Count the occurrences of each type in the current chunk
        type_chunk_counts = chunk[type_column].value_counts()

        # Aggregate the counts with the overall type_counts dictionary
        for typ, count in type_chunk_counts.items():
            type_counts[typ] = type_counts.get(typ, 0) + count

        pbar.update()

# Print the speed counts
print("Speed Counts:")
for speed, count in speed_counts.items():
    print(f"Speed: {speed}, Count: {count}")

# Print the type counts
print("Type Counts:")
for typ, count in type_counts.items():
    print(f"Type: {typ}, Count: {count}")


Processing CSV: 48chunk [00:50,  1.06s/chunk]                   

Speed Counts:
Speed: Variable_speed, Count: 47185920
Type Counts:
Type: PGB, Count: 23592960
Type: RGB, Count: 23592960
